In [2]:
from augur.io import read_sequences
from collections import OrderedDict
import pandas as pd
from augur.utils import read_node_data

In [3]:
sequences_by_name = OrderedDict()

for sequence in read_sequences("../ha-na-nextstrain/results/aligned_ha.fasta"):
    sequences_by_name[sequence.id] = str(sequence.seq)

sequence_names = list(sequences_by_name.keys())

In [4]:
sequences = pd.DataFrame.from_dict(sequences_by_name, orient='index')
sequences = sequences[0].apply(lambda x: pd.Series(list(x)))
consensus = []
for (name, data) in sequences.iteritems():
    consensus.append(data.value_counts().idxmax())
#for sequence in list(sequences_by_name.values()):
#    for character in sequence:
string_consensus = "".join(consensus)   

In [5]:
string_consensus

'ATGAAGACTATCATTGCTTTGAGCTACATTCTATGTCTGGTTTTCGCTCAAAAAATTCCTGGAAATGACAATAGCACGGCAACGCTGTGCCTTGGGCACCATGCAGTACCAAACGGAACGATAGTGAAAACAATCACAAATGACCGAATTGAAGTTACTAATGCTACTGAGTTGGTTCAGAATTCCTCAATAGGTGAAATATGCGACAGTCCTCATCAGATCCTTGATGGAGAAAACTGCACACTAATAGATGCTCTATTGGGAGACCCTCAGTGTGATGGCTTTCAAAATAAGAAATGGGACCTTTTTGTTGAACGAAGCAAAGCCTACAGCAACTGTTACCCTTATGATGTGCCGGATTATGCCTCCCTTAGGTCACTAGTTGCCTCATCCGGCACACTGGAGTTTAACAATGAAAGCTTCAATTGGACTGGAGTCACTCAAAACGGAACAAGTTCTGCTTGCATAAGGAGATCTAGTAGTAGTTTCTTTAGTAGATTAAATTGGTTGACCCACTTAAACTACACATATCCAGCATTGAACGTGACTATGCCAAACAATGAACAATTTGACAAATTGTACATTTGGGGGGTTCACCACCCGGGTACGGACAAGGACCAAATCTTCCTGTATGCTCAATCATCAGGAAGAATCACAGTATCTACCAAAAGAAGCCAACAAGCTGTAATCCCAAATATCGGATCTAGACCCAGAATAAGGGATATCCCTAGCAGAATAAGCATCTATTGGACAATAGTAAAACCGGGAGACATACTTTTGATTAACAGCACAGGGAATCTAATTGCTCCTAGGGGTTACTTCAAAATACGAAGTGGGAAAAGCTCAATAATGAGATCAGATGCACCCATTGGCAAATGCAAGTCTGAATGCATCACTCCAAATGGAAGCATTCCCAATGACAAACCATTCCAAAATGTAAACAGGATCACATACGGGGCCTGTCCCAGATATGTTAAGCATAGCACTCTGAAATTGGCA

In [6]:
metadata = pd.read_csv("../ha-na-nextstrain/results/full_HDBSCAN_metadata.csv")

In [7]:
node_data = read_node_data("../ha-na-nextstrain/results/mccs.json")

In [8]:
clade_annotations = pd.DataFrame([
    {"strain": strain, "clade": annotations["MCC"]}
    for strain, annotations in node_data["nodes"].items()
    if strain in sequences_by_name
])
clade_annotations = clade_annotations.set_index("strain")

In [9]:
clade_annotations.index

Index(['A/Alabama/01/2017', 'A/Alabama/03/2017', 'A/Alabama/04/2017',
       'A/Alabama/06/2017', 'A/Alabama/08/2017', 'A/Alabama/09/2017',
       'A/Alabama/10/2017', 'A/Alabama/13/2016', 'A/Alabama/14/2016',
       'A/Alabama/15/2016',
       ...
       'A/Wyoming/03/2017', 'A/Wyoming/08/2017', 'A/Wyoming/10/2017',
       'A/Wyoming/12/2017', 'A/Wyoming/15/2016', 'A/Wyoming/16/2017',
       'A/Wyoming/17/2017', 'A/Wyoming/21/2017', 'A/Wyoming/27/2017',
       'A/Wyoming/29/2016'],
      dtype='object', name='strain', length=1643)

In [12]:
clade = clade_annotations.groupby(["clade"])
list_clades = [clade.get_group(x) for x in clade.groups]
# merge strain information (make a dictionary)

In [20]:
sequences_by_name[['A/Texas/80/2017', 'A/Wyoming/17/2017']]

TypeError: unhashable type: 'list'

In [35]:
for clades in list_clades:
    sequences = []
    for index, row in clades.iterrows():
        sequences.append(sequences_by_name[index])
    clades["sequence"] = sequences

/home/nandsra/anaconda3/envs/cartography/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [61]:
total_consensus_per_group = []
for clades in list_clades:
    consensus = []
    for (name, data) in clades[["sequence"]].iteritems():
        #consensus.append(data.value_counts().idxmax())
        #print(name)
        split_data = pd.Series(data[0]).apply(lambda x: pd.Series(list(x)))
        consensus.append(split_data.value_counts().idxmax())
    str_consensus = "".join(list(consensus[0]))
    total_consensus_per_group.append(str_consensus)

In [69]:
grouped_strains = []
for group in list(clade_annotations.groupby(["clade"]).groups.values()):
    strains = []
    for strain in group:
        strains.append(sequences_by_name[strain])
    grouped_strains.append(strains)

In [71]:
for strains in grouped_strains:
    strains = [strain.split() for strain in strains]
    pd.Dataframe.from_list(strains)
    #consensus = []
    #for (name, data) in sequences.iteritems():
    #    consensus.append(data.value_counts().idxmax())
    #for sequence in list(sequences_by_name.values()):
    #    for character in sequence:
    #string_consensus = "".join(consensus)   

AttributeError: module 'pandas' has no attribute 'Dataframe'

In [65]:
len(total_consensus_per_group) == len(clade.groups)

True

In [69]:
clade_consensus_strain = dict(zip(clade.groups, total_consensus_per_group))

In [70]:
clade_consensus_strain

{'MCC_0': 'ATGAAGACTATCATTGCTTTGAGCTACATTCTATGTCTGGTTTTCGCTCAAAAAATTCCTGGAAATGACAATAGCACGGCAACGCTGTGCCTTGGGCACCATGCAGTACCAAACGGAACGATAGTGAAAACAATCACAAATGACCGAATTGAAGTTACTAATGCTACTGAGTTGGTTCAGAATTCCTCAATAGGTGAAATATGCGACAGTCCTCATCAGATCCTTGATGGAGAAAACTGCACACTAATAGATGCTCTATTGGGAGACCCTCAGTGTGATGGCTTTCAAAATAAGAAATGGGACCTTTTTGTTGAACGAAGCAAAGCCTACAGCAACTGTTACCCTTATGATGTGCCGGATTATGCCTCCCTTAGGTCACTAGTTGCCTCATCCGGCACACTGGAGTTTAACAATGAAAGCTTCAATTGGACTGGAGTCAAACAAAACGGAACAAGTTCTGCTTGTATAAGGAAATCTAGTAGTAGTTTCTTTAGTAGGTTAAATTGGTTGACCCACTTAAACTACACATATCCAGCATTGAACGTGACTATGCCAAACAATGAACAATTTGACAAATTGTACATTTGGGGGGTTCACCACCCGGGTACGGACAAGGACCAAATCTTCCTGTATGCTCAATCATCAGGAAGAATCACAGTATCTACCAAAAGAAGCCAACAAGCTGTAATCCCAAATATCGGATCCAGACCCAGAATAAGGGATATCCCTAGCAGAATAAGCATCTATTGGACAATAGTAAAACCGGGAGACATACTTTTGATTAACAGCACAGGGAATCTAATTGCTCCTAGGGGTTACTTCAAAATACAAAGTGGGAAAAGCTCAATAATGAGATCAGATGCACCCATTGGCAAATGCAAGTCTGAATGCATCACTCCAAATGGAAGCATTCCCAATGACAAACCATTCCAAAATGTAAACAGGATCACATACGGGGCCTGTCCCAGATATGTTAAGCATAGCACTCT